<a href="https://colab.research.google.com/github/RafaelCaballero/Julio25/blob/main/code/proyecto_regresion_completado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python

## Regresión sobre datos bursátiles

### Rafael Caballero

#### Paso 0: importar librerías y cargar y preprocesar datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf
import pandas as pd

# Lista de tickers:
valores = ["inditex","iberdrola", "santander", "BBVA", "naturgy"]
tickers = ['ITX.MC', 'IBE.MC',    'SAN.MC', 'BBVA.MC', 'NTGY.MC']

# Fechas de inicio y fin
start_date = '2022-01-01'
end_date = '2025-07-07'

# Descargar los datos de Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Crear un DataFrame vacío para almacenar los valores OPEN
close_data = pd.DataFrame()

# Extraer los valores OPEN de cada ticker y añadirlos al DataFrame
for i,ticker in enumerate(tickers):
    nombre = valores[i]
    close_data[nombre] = data[ticker]['Close']


# incremento en porcentaje con respecto al día anterior
roi = close_data.pct_change()*100
roi.columns = ["r_"+c for c in close_data.columns]

df = pd.concat([close_data,roi],axis=1)

# añadimos el día de la semana
df['dia_semana'] = roi.index.to_series().dt.day_name()

# mejor en castellano
days_translation = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
df['dia_semana'] = df['dia_semana'].map(days_translation)

# Establecer el índice como una versión formateada de la fecha
df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')

df


/tmp/ipython-input-1-4278836117.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
[*********************100%***********************]  5 of 5 completed


,inditex,iberdrola,santander,BBVA,naturgy,r_inditex,r_iberdrola,r_santander,r_BBVA,r_naturgy,dia_semana
Date,,,,,,,,,,,
2022-01-03,26.194273,8.764927,2.562269,4.118003,23.679726,NaN,NaN,NaN,NaN,NaN,Lunes
2022-01-04,26.057180,8.714579,2.633479,4.222126,23.588682,-0.523369,-0.574429,2.779202,2.528497,-0.384479,Martes
2022-01-05,26.011480,8.655838,2.637821,4.211949,23.621790,-0.175384,-0.674050,0.164871,-0.241043,0.140354,Miércoles
2022-01-06,25.837828,8.601293,2.679505,4.273015,23.737663,-0.667600,-0.630157,1.580249,1.449820,0.490536,Jueves
2022-01-07,25.280304,8.605489,2.702953,4.374007,23.605232,-2.157781,0.048785,0.875060,2.363477,-0.557894,Viernes
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,44.180000,15.887238,7.027000,13.055000,27.000000,0.249604,-0.214391,-0.142109,0.345888,0.671142,Lunes
2025-07-01,44.369999,15.979888,6.965000,12.850000,27.219999,0.430056,0.583175,-0.882308,-1.570279,0.814812,Martes
2025-07-02,43.990002,15.887238,7.120000,13.035000,27.160000,-0.856428,-0.579794,2.225409,1.439685,-0.220424,Miércoles


In [2]:
df = df.iloc[1:]
df

,inditex,iberdrola,santander,BBVA,naturgy,r_inditex,r_iberdrola,r_santander,r_BBVA,r_naturgy,dia_semana
Date,,,,,,,,,,,
2022-01-04,26.057180,8.714579,2.633479,4.222126,23.588682,-0.523369,-0.574429,2.779202,2.528497,-0.384479,Martes
2022-01-05,26.011480,8.655838,2.637821,4.211949,23.621790,-0.175384,-0.674050,0.164871,-0.241043,0.140354,Miércoles
2022-01-06,25.837828,8.601293,2.679505,4.273015,23.737663,-0.667600,-0.630157,1.580249,1.449820,0.490536,Jueves
2022-01-07,25.280304,8.605489,2.702953,4.374007,23.605232,-2.157781,0.048785,0.875060,2.363477,-0.557894,Viernes
2022-01-10,25.298584,8.550025,2.705558,4.366179,23.770771,0.072309,-0.644516,0.096392,-0.178961,0.701280,Lunes
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,44.180000,15.887238,7.027000,13.055000,27.000000,0.249604,-0.214391,-0.142109,0.345888,0.671142,Lunes
2025-07-01,44.369999,15.979888,6.965000,12.850000,27.219999,0.430056,0.583175,-0.882308,-1.570279,0.814812,Martes
2025-07-02,43.990002,15.887238,7.120000,13.035000,27.160000,-0.856428,-0.579794,2.225409,1.439685,-0.220424,Miércoles


In [3]:
df.to_excel("datos_preparados.xlsx",index=False)

In [4]:
df["r_BBVA_mañana"] = df["r_BBVA"].shift(-1)
df

/tmp/ipython-input-4-3679338387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["r_BBVA_mañana"] = df["r_BBVA"].shift(-1)


,inditex,iberdrola,santander,BBVA,naturgy,r_inditex,r_iberdrola,r_santander,r_BBVA,r_naturgy,dia_semana,r_BBVA_mañana
Date,,,,,,,,,,,,
2022-01-04,26.057180,8.714579,2.633479,4.222126,23.588682,-0.523369,-0.574429,2.779202,2.528497,-0.384479,Martes,-0.241043
2022-01-05,26.011480,8.655838,2.637821,4.211949,23.621790,-0.175384,-0.674050,0.164871,-0.241043,0.140354,Miércoles,1.449820
2022-01-06,25.837828,8.601293,2.679505,4.273015,23.737663,-0.667600,-0.630157,1.580249,1.449820,0.490536,Jueves,2.363477
2022-01-07,25.280304,8.605489,2.702953,4.374007,23.605232,-2.157781,0.048785,0.875060,2.363477,-0.557894,Viernes,-0.178961
2022-01-10,25.298584,8.550025,2.705558,4.366179,23.770771,0.072309,-0.644516,0.096392,-0.178961,0.701280,Lunes,0.322731
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,44.180000,15.887238,7.027000,13.055000,27.000000,0.249604,-0.214391,-0.142109,0.345888,0.671142,Lunes,-1.570279
2025-07-01,44.369999,15.979888,6.965000,12.850000,27.219999,0.430056,0.583175,-0.882308,-1.570279,0.814812,Martes,1.439685
2025-07-02,43.990002,15.887238,7.120000,13.035000,27.160000,-0.856428,-0.579794,2.225409,1.439685,-0.220424,Miércoles,1.687766


In [5]:
# quitamos la última porque la última no tiene mañana
df = df.iloc[:-1]

## Empezamos la regresión

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

# Paso 1: Dividir las columnas en X e y e.
# Nuestro objetivo será deducir la columna MAT (y) desde dos columnas: SCI y REA, que serán las X
XColumns = ['inditex', 'iberdrola', 'santander', 'BBVA', 'naturgy', 'r_inditex',
       'r_iberdrola', 'r_santander', 'r_BBVA', 'r_naturgy']
yColumn = "r_BBVA_mañana"  # la y, siempre una única columna
# dividimos en dos; X será un dataframe, y una columna (tipo Series)
X = df[XColumns]
y = df[yColumn]


# número de divisiones para validación cruzada, e int. de confianza
repeticiones   = 100  # número de repeticiones del experimento
divisiones = 10  # número de divisiones cada vez

# error a manejar
medidor_error=make_scorer(mean_squared_error, greater_is_better=False)

In [7]:
repartidor   = RepeatedKFold(n_splits=divisiones, n_repeats=repeticiones)
método = LinearRegression()
cv_res  = cross_validate(método, X, y,
                         cv=repartidor,
                         return_estimator=True,  # devuelve los 1000 modelos!
                         scoring=medidor_error)
# ojo porque los valores obtenidos son negativos
mse_neg = cv_res["test_score"]
rmse    = np.sqrt(-mse_neg)
rmse_final = rmse.mean()
print(f"RMSE: {rmse_final:.3f}")

RMSE: 1.926


### Prueba base

¿Qué error tendríamos utilizando para predecir los rendimientos de mañana el valor de hoy?

In [9]:
y_true = df[yColumn]
y_pred = df["r_BBVA"]

# calculamso el RMSE
np.sqrt(mean_squared_error(y_true,y_pred))

np.float64(2.6580138383896865)

### Selección de atributos

In [10]:
XColumns = [ 'r_iberdrola',  'r_BBVA', 'r_naturgy']
yColumn = "r_BBVA_mañana"  # la y, siempre una única columna
# dividimos en dos; X será un dataframe, y una columna (tipo Series)
X = df[XColumns]
y = df[yColumn]


# número de divisiones para validación cruzada, e int. de confianza
repeticiones   = 100  # número de repeticiones del experimento
divisiones = 10  # número de divisiones cada vez

# error a manejar
medidor_error=make_scorer(mean_squared_error, greater_is_better=False)

repartidor   = RepeatedKFold(n_splits=divisiones, n_repeats=repeticiones)
método = LinearRegression()
cv_res  = cross_validate(método, X, y,
                         cv=repartidor,
                         return_estimator=True,  # devuelve los 1000 modelos!
                         scoring=medidor_error)
# ojo porque los valores obtenidos son negativos
mse_neg = cv_res["test_score"]
rmse    = np.sqrt(-mse_neg)
rmse_final = rmse.mean()
print(f"RMSE: {rmse_final:.3f}")

RMSE: 1.912


### Lasso y Ridge

Variantes de la regresión lineal que se adaptan mejor a datos erróneos

In [13]:
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

ridge = make_pipeline(
    StandardScaler(),                # muy importante
    Ridge(alpha=1.0)
)
lasso = make_pipeline(
    StandardScaler(),
    Lasso(alpha=0.1, max_iter=10000)
)

def cv_rmse(modelo, nombre):
    res = cross_validate(modelo, X, y,
                         cv=repartidor,
                         scoring=medidor_error)
    rmse = np.sqrt(-res['test_score']).mean()
    print(f"RMSE {nombre}: {rmse:.3f}")

cv_rmse(ridge, 'Ridge α=1.0')
cv_rmse(lasso, 'Lasso α=0.1')

RMSE Ridge α=1.0: 1.911
RMSE Lasso α=0.1: 1.903


In [ ]:
alphas_ridge = np.logspace(-3, 3, 50)   # 0.001 … 1000
alphas_lasso = np.logspace(-3, 1, 40)   # 0.001 … 10

ridge_cv = make_pipeline(
    StandardScaler(),
    RidgeCV(alphas=alphas_ridge,
            cv=25,
            scoring='neg_mean_squared_error')
)
lasso_cv = make_pipeline(
    StandardScaler(),
    LassoCV(alphas=alphas_lasso,
            cv=25,
            max_iter=10000)
)

cv_rmse(ridge_cv, 'RidgeCV (α óptimo)')
cv_rmse(lasso_cv, 'LassoCV (α óptimo)')